In [2]:
import os
import time
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
import FinNews as fn
import re

# Dash modules
import dash
import dash_table
import dash_core_components as dcc
import dash_bootstrap_components as dbc
import dash_html_components as html
from jupyter_dash import JupyterDash
import plotly.express as px
import plotly.graph_objects as go

# Set up jupyter proxy
JupyterDash.infer_jupyter_proxy_config() 

%matplotlib inline

In [17]:
# Getting all file paths

path = r'assets/historical-symbols' # use your path
all_files = glob.glob(path + "/*.csv")

# Creating list to append all ticker dfs to
li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

# Concat all ticker dfs 
stock_df = pd.concat(li, axis=0, ignore_index=True)

stock_df['Date'] = pd.to_datetime(stock_df['Date'])

# Creating Moving Average Technical Indicator
# Using this aritcle https://towardsdatascience.com/building-a-comprehensive-set-of-technical-indicators-in-python-for-quantitative-trading-8d98751b5fb

stock_df['SMA_5'] = stock_df.groupby('ticker')['Close'].transform(lambda x: x.rolling(window = 5).mean())
stock_df['SMA_15'] = stock_df.groupby('ticker')['Close'].transform(lambda x: x.rolling(window = 15).mean())
stock_df['SMA_ratio'] = stock_df['SMA_15'] / stock_df['SMA_5']

# Bollinger bands
stock_df['15MA'] = stock_df.groupby('ticker')['Close'].transform(lambda x: x.rolling(window=15).mean())
stock_df['SD'] = stock_df.groupby('ticker')['Close'].transform(lambda x: x.rolling(window=15).std())
stock_df['upperband'] = stock_df['15MA'] + 2*stock_df['SD']
stock_df['lowerband'] = stock_df['15MA'] - 2*stock_df['SD']

available_indicators1 = list(stock_df['ticker'].unique())

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




In [19]:
stock_df.sector.unique()

array(['Technology', 'Industrials', 'Financial Services', 'Healthcare',
       'Consumer Cyclical', 'Energy', 'Basic Materials', 'Real Estate',
       'Communication Services', 'Consumer Defensive', 'Utilities'],
      dtype=object)

In [3]:
title_style = {'display': 'inline-block',
               'textAlign':'left',
               'verticalAlign':'center',
               'lineHeight':2.5,
               'height':75,
               'border': 'thick black solid',\
               'width':'85%',
               'backgroundColor': 'rgb(212, 150, 18)'}

title_link_style = {'display': 'inline-block',
               'textAlign':'center',
               'vertical-align':'center',
               'lineHeight':5,
               'height':75,
               'width':'10%',
               'float':'right',
               'border': 'thick black solid',\
               'fontSize':12,
               'backgroundColor': 'rgb(212, 150, 18)'}


tab_style = {'display': 'inline-block',
             'textAlign':'center',
             'verticalAlign':'top',
             'width':'15%',
             'border': 'thin lightgrey solid',
             'height':20}

blank_tab_style = {'display': 'inline-block',
                   'width':'15%',
                   'textAlign':'center',
                   'verticalAlign':'top',
                   'float':'right',
                   'border': 'thin lightgrey solid',
                   'fontSize':10,
                   'height':20}

ticker_style = {'display': 'inline-block',
                'textAlign':'center',
                'vertical-align':'top',
                'width':'20%',
                'float':'right',
                'border':'thin lightgrey solid',
                'height':300}

chart_style = {'display':'inline-block',
               'textAlign':'center',
               'vertical-align':'top',
               'width':'60%',
               'float':'middle',
               'textAlign':'center',
               'border':'thin lightgrey solid',
               'height':300}

portfolio_style = {'display': 'inline-block',
                'textAlign':'center',
                'vertical-align':'top',
                'width':'20%',
                'float':'left',
                'border':'thin lightgrey solid',
                'height':300}

chart_style_b = {'display':'inline-block',
               'textAlign':'center',
               'vertical-align':'top',
               'width':'60%',
               'float':'middle',
               'textAlign':'center',
               'border':'thin lightgrey solid',
               'height':200}

portfolio_style_b = {'display': 'inline-block',
                'textAlign':'left',
                'fontSize':12,
                'vertical-align':'top',
                'width':'20%',
                'float':'left',
                'border':'thin lightgrey solid',
                'height':200}

news_style = {'fontSize':11,
              'color':'white',
              'textAlign':'left',
              'height':'auto',
#               'height':27,
              'margin':3,
              "lineHeight":1.5,
              'float':'middle'}

news_style_b = {'display': 'inline-block',
                'textAlign':'left',
                'vertical-align':'top',
                'width':'20%',
                'float':'right',
                'border':'thin lightgrey solid',
                'height':300,'margin':0,'overflowY':'scroll',
                'color':'white',
                'backgroundColor':'black',
                'borderRadius': '.4rem'}



#                'vertical-align':'middle',

In [5]:
app = JupyterDash(__name__,suppress_callback_exceptions=True)

server = app.server

app.layout = html.Div([

    html.Div([
        html.H1('Dashboard Title',style=title_style),
        html.H3('Dashboard Info Link',style=title_link_style)
        ]),

    dcc.Tabs(id='tabs-example', value='tab-1', children=[
        dcc.Tab(label='Portfolio Performance', value='tab-1'),
        dcc.Tab(label='Model Performance', value='tab-2'),
        dcc.Tab(label='Tab three', value='tab-3')
    ],style={'width':'50%','height':50}),
    html.Div(id='tabs-example-content')
])

@app.callback(dash.dependencies.Output('tabs-example-content', 'children'),
              [dash.dependencies.Input('tabs-example', 'value')])
def render_content(tab):
    if tab == 'tab-1':
        return html.Div([
            
                html.Div([
                    dcc.Dropdown(id='ticker',
                    options=[{'label': i, 'value': i} for i in available_indicators1],
                    value='CSCO') # the default is code_module AAA
                    ]),
                
                html.Div([
                    html.H2('Portfolio Performance',style=portfolio_style),
                    dcc.Graph(id='price_chart',style=chart_style),
                    html.Div(id='news_list',children=news_info,style=news_style_b)
                    ]),
                    
                html.Div([
                    html.H2('Other Portfolio Statistics',style=portfolio_style_b),
                    html.H2('News Info',style=chart_style_b),
                    ],style={'height':200})
    
                ])
                
    elif tab == 'tab-2':
        return html.Div([
            html.H3('Tab content 2')
        ])
    
# Callback to connect input(s) to output(s)
@app.callback(dash.dependencies.Output('price_chart','figure'),
    [dash.dependencies.Input('ticker','value')])

# Step 3: Define the graph with plotly express
def update_ticker(ticker):
    
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(x=stock_df[stock_df['ticker']==ticker]['Date'],
                             y=stock_df[stock_df['ticker']==ticker]['Close'],
                            line={"color": "#228B22"},
                            mode="lines"))

    fig.update_layout(title_text=f'{ticker} Closing Price',title_x=0.5,
                         template="plotly_dark",font=dict(size=10),xaxis_showgrid=False,
                         yaxis_title="Closing Price",margin={"r": 20, "t": 35, "l": 20, "b": 10})

    return fig

# Creating callback to get news when ticker changes
@app.callback(dash.dependencies.Output('news_list', 'children'),
              [dash.dependencies.Input('ticker', 'value')])
def update_news(ticker):
    
    seeking_alpha = fn.SeekingAlpha(topics=['$'+ticker], save_feeds=True)
    
    news = seeking_alpha.get_news()
    
#     title_strings = [re.sub('[^A-Za-z0-9,\s]+', '', item['title']).lower().replace(" ", "-") for item in news[:5]]
#     url_ids = [re.sub('MarketCurrent:','news/',item) for item in news[:5]]
    
    news_info = html.Div(f'News for {ticker}',style={'backgroundColor':'gray'}),\
                    dbc.ListGroup([
                        dbc.ListGroupItem([html.Div([
                                        html.A(html.P(news[0]['title'],style=news_style),\
                                        href=(news[0]['link'])),\
                                        html.A(html.P(news[0]['published'],style=news_style))
                                        ])
                                  ],color='gray'),\
                    dbc.ListGroupItem([html.Div([
                                        html.A(html.P(news[1]['title'],style=news_style),\
                                        href=(news[1]['link'])),\
                                        html.A(html.P(news[1]['published'],style=news_style))
                                        ])
                                  ],color='gray'),\
                    dbc.ListGroupItem([html.Div([
                                        html.A(html.P(news[2]['title'],style=news_style),\
                                        href=(news[2]['link'])),\
                                        html.A(html.P(news[2]['published'],style=news_style))
                                        ])
                                  ],color='gray'),\
                    dbc.ListGroupItem([html.Div([
                                        html.A(html.P(news[3]['title'],style=news_style),\
                                        href=(news[3]['link'])),\
                                        html.A(html.P(news[3]['published'],style=news_style))
                                        ])
                                  ],color='gray'),\
                    dbc.ListGroupItem([html.Div([
                                        html.A(html.P(news[4]['title'],style=news_style),\
                                        href=(news[4]['link'])),\
                                        html.A(html.P(news[4]['published'],style=news_style))
                                        ])
                                  ],color='gray'),\
                    ],flush=True)
                    
    return news_info

if __name__ == '__main__':
    app.run_server(debug=True,port=8051)

Dash app running on http://127.0.0.1:8051/


### Notes
1. How do I set a relative height
2. Could add ticker - https://community.plotly.com/t/strip-ticker-label/48348/6


In [15]:
import dash
from dash.dependencies import Input, Output
import dash_html_components as html

app = JupyterDash(__name__,suppress_callback_exceptions=True)

server = app.server

app.layout = html.Div([

    html.H2('Thank Mbkupfer for this modal'),

    html.Div([  # modal div
        html.Div([  # content div
            html.Div([
                'This is the content of the modal',

            ]),

            html.Hr(),
            html.Button('Close', id='modal-close-button')
        ],
            style={'textAlign': 'center', },
            className='modal-content',
        ),
    ],
        id='modal',
        className='modal',
        style={"display": "block"},
    )
])

@app.callback(Output('modal', 'style'),
              [Input('modal-close-button', 'n_clicks')])
def close_modal(n):
    if (n is not None) and (n > 0):
        return {"display": "none"}

if __name__ == '__main__':
    app.run_server(debug=True,port=8090)

KeyboardInterrupt: 

In [20]:
html.Div([
        # Adding drop down to filter by ticker
        dcc.Dropdown(id='ticker',
            options=[{'label': i, 'value': i} for i in list(stock_df['ticker'].unique())],
            value='CSCO',style={'margin':'5px','width':'40%'}), # the default is code_module AAA

        dcc.Dropdown(id='industry_ticker',
            options=[{'label': i, 'value': i} for i in list(stock_df['sector'].unique())],
            value='Technology',style={'margin':'5px','width':'40%'}) # the default is code_module AAA
            ])

Div([Dropdown(id='ticker', options=[{'label': 'CSCO', 'value': 'CSCO'}, {'label': 'UAL', 'value': 'UAL'}, {'label': 'TROW', 'value': 'TROW'}, {'label': 'ISRG', 'value': 'ISRG'}, {'label': 'NVR', 'value': 'NVR'}, {'label': 'PRGO', 'value': 'PRGO'}, {'label': 'TPR', 'value': 'TPR'}, {'label': 'DVN', 'value': 'DVN'}, {'label': 'CE', 'value': 'CE'}, {'label': 'MRO', 'value': 'MRO'}, {'label': 'BA', 'value': 'BA'}, {'label': 'VRTX', 'value': 'VRTX'}, {'label': 'GILD', 'value': 'GILD'}, {'label': 'NLSN', 'value': 'NLSN'}, {'label': 'EQIX', 'value': 'EQIX'}, {'label': 'TER', 'value': 'TER'}, {'label': 'MDT', 'value': 'MDT'}, {'label': 'V', 'value': 'V'}, {'label': 'QRVO', 'value': 'QRVO'}, {'label': 'A', 'value': 'A'}, {'label': 'FOX', 'value': 'FOX'}, {'label': 'FLT', 'value': 'FLT'}, {'label': 'MO', 'value': 'MO'}, {'label': 'SWKS', 'value': 'SWKS'}, {'label': 'ENPH', 'value': 'ENPH'}, {'label': 'MCHP', 'value': 'MCHP'}, {'label': 'CDNS', 'value': 'CDNS'}, {'label': 'WLTW', 'value': 'WLTW'}

In [22]:
[{'label': i, 'value': i} for i in ['Stocks','Sector']]

[{'label': 'Stocks', 'value': 'Stocks'},
 {'label': 'Sector', 'value': 'Sector'}]